In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import tensorflow as tf
from keras import backend
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Activation, Bidirectional, LSTM, Dropout, Input
from keras.optimizers import RMSprop
import pickle
import heapq
print(tf.__version__)
print(tf.config.list_physical_devices())

2.9.2
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
df_wiki = pd.read_csv('/content/drive/Shareddrives/ANLY 580 project /wiki_100000.csv')
df_wiki

,Unnamed: 0,id,url,title,text
0,0,1658014,https://en.wikipedia.org/wiki/Adrienne%20Mayor,Adrienne Mayor,Adrienne Mayor (born 1946) is a historian of a...
1,1,415109,https://en.wikipedia.org/wiki/Jo%20Stafford,Jo Stafford,"Jo Elizabeth Stafford (November 12, 1917July 1..."
2,2,733308,https://en.wikipedia.org/wiki/Milan%20Rapai%C4%87,Milan Rapaić,"Milan ""Miki"" Rapaić (born 16 August 1973) is a..."
3,3,2597099,https://en.wikipedia.org/wiki/Windsor%20North%...,Windsor North School,Windsor North School is a primary school in In...
4,4,690250,https://en.wikipedia.org/wiki/List%20of%20rive...,List of rivers of Missouri,List of rivers in Missouri (U.S. state).\n\nBy...
...,...,...,...,...,...
99995,99995,670737,https://en.wikipedia.org/wiki/Standesamt%20Ade...,Standesamt Adelnau,Standesamt Adelnau was one of the civil regist...
99996,99996,580501,https://en.wikipedia.org/wiki/Siaka%20Stevens,Siaka Stevens,Siaka Probyn Stevens (24 August 1905 – 29 May ...
99997,99997,2214203,https://en.wikipedia.org/wiki/Malitbog,Malitbog,Malitbog is the name of several places in the ...
99998,99998,2177759,https://en.wikipedia.org/wiki/Calyx%20%28music...,Calyx (musician),"Calyx is a British drum and bass act, speciali..."


In [ ]:
def text_cleaner(text):
    clean_text = text.lower()
    url = r'''(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'''
    clean_text = re.sub(url, '', clean_text)
    clean_text = re.sub(r'\([^)]*\)', '', clean_text)
    clean_text = re.sub(r'\[|\]', '', clean_text)
    clean_text = re.sub(r'\“|\”','', clean_text)
    clean_text = re.sub(r'\"','', clean_text)
    clean_text = re.sub(r"(?<=\d),(?=\d)", "", clean_text)
    clean_text = re.sub(r"[\-\—]", " ", clean_text)
    clean_text = re.sub(r"\d+", "number", clean_text)
    clean_text = re.sub(r"'s", "", clean_text)
    clean_text = re.sub(r"[^a-zA-Z0-9 \,\.\!\?]", "", clean_text)
    clean_text = re.sub(r"\s+", " ", clean_text)
    return clean_text

In [ ]:
set_of_chars = sorted([' ', '!', ',', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
  'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
char_to_index = dict((c, i) for i, c in enumerate(set_of_chars))
index_to_char = dict((i, c) for i, c in enumerate(set_of_chars))
len(set_of_chars)

31

In [ ]:
document_size = 100
sequence_length = 32
next_length = 10
step = 4
def generator(data, document_size, sequence_length, next_length, step, batch_size=128):
  for i in range(0, len(data), document_size):
    documents = data[i : i + batch_size]["text"].map(text_cleaner).tolist()
    sentences = []
    next_chars = []
    for text in documents:
      for i in range(0, len(text) - sequence_length, step):
        if i + sequence_length + next_length < len(text):
          sentences.append(text[i: i + sequence_length])
          next_chars.append(text[i + sequence_length: i + sequence_length + next_length])
    for i in range(0, len(sentences), batch_size):
      current_sentence = sentences[i : i + batch_size]
      current_next_chars = next_chars[i : i + batch_size] 
      X = np.zeros((batch_size, sequence_length, len(set_of_chars)), dtype=bool)
      y = np.zeros((batch_size, next_length, len(set_of_chars)), dtype=bool)
      y_2 = np.zeros((batch_size, next_length, len(set_of_chars)), dtype=bool)
      for i, sentence in enumerate(current_sentence):
        for t, char in enumerate(sentence):
          X[i, t, char_to_index[char]] = 1
        for t, char in enumerate(current_next_chars[i]):
          y[i, t, char_to_index[char]] = 1
          if t > 0:
            y_2[i, t-1, char_to_index[char]] = 1
      yield [X, y], y_2

In [ ]:
backend.clear_session()
batch_size = 128
epochs = 10
latent_dim = 128

encoder_inputs = Input(shape=(None,len(set_of_chars)))
encoder = LSTM(latent_dim,return_state=True)
encoder_outputs , state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h,state_c]


decoder_inputs = Input(shape=(None,len(set_of_chars)))
decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_ouputs,_,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)

decoder_dense = Dense(len(set_of_chars), activation='softmax')
decoder_ouputs = decoder_dense(decoder_ouputs)
# At this time, the model is basically a seq2seq process, 
# and the loss function is defined as categorical cross entropy, 
# because the essence is the process of completing category division.
model = Model([encoder_inputs,decoder_inputs],decoder_ouputs)
optimizer = RMSprop(learning_rate=0.01)
model.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 31)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 31)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 128),        81920       ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

In [ ]:
train_gen = generator(data=df_wiki, document_size=document_size, sequence_length=sequence_length, next_length=next_length, step=step, batch_size=128)

In [ ]:
h=model.fit(train_gen, epochs=10, batch_size=batch_size, steps_per_epoch=2500)

Epoch 1/10
2500/2500 [==============================] - 32s 11ms/step - loss: 1.8458 - accuracy: 0.4130
Epoch 2/10
2500/2500 [==============================] - 31s 12ms/step - loss: 1.6567 - accuracy: 0.4699
Epoch 3/10
2500/2500 [==============================] - 30s 12ms/step - loss: 1.6509 - accuracy: 0.4698
Epoch 4/10
2500/2500 [==============================] - 26s 11ms/step - loss: 1.5945 - accuracy: 0.4873
Epoch 5/10
2500/2500 [==============================] - 26s 11ms/step - loss: 1.6233 - accuracy: 0.4849
Epoch 6/10
2500/2500 [==============================] - 28s 11ms/step - loss: 1.6021 - accuracy: 0.4862
Epoch 7/10
2500/2500 [==============================] - 27s 11ms/step - loss: 1.6726 - accuracy: 0.4643
Epoch 8/10
2500/2500 [==============================] - 27s 11ms/step - loss: 1.6053 - accuracy: 0.4856
Epoch 9/10
2500/2500 [==============================] - 26s 10ms/step - loss: 1.6675 - accuracy: 0.4641
Epoch 10/10
2500/2500 [==============================] - 27s 11m

In [ ]:
encoder_model = Model(encoder_inputs,encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs,state_h,state_c = decoder_lstm(
        decoder_inputs,initial_state = decoder_states_inputs
)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')

In [ ]:
def encode_input(text):
    encoded_text = np.zeros((1, sequence_length, len(set_of_chars)))
    for t, char in enumerate(text):
      encoded_text[0, t, char_to_index[char]] = 1    
    return encoded_text

def top_prediction(preds, n):
    preds = np.array(preds)
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.argsort(preds)[0][0][:len(preds)-n-2:-1]

In [ ]:
def decode_sequence(input_seq):
  states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, len(set_of_chars)))

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # Sample a token
    sampled_token_index = top_prediction(output_tokens, 5)[0]
    sampled_char = index_to_char[sampled_token_index]
    decoded_sentence += sampled_char

    # Exit condition: either hit max length
    # or find stop character.
    if (sampled_char == ' ' or len(decoded_sentence) > next_length):
      stop_condition = True
      # Update the target sequence (of length 1).
      target_seq = np.zeros((1, 1, len(set_of_chars)))
      target_seq[0, 0, sampled_token_index] = 1.

      # Update states
      states_value = [h, c]
    
  return decoded_sentence

In [ ]:
text = "Machine learning is a field of inquiry devoted to understanding and building methods that lear"
encode_text = encode_input(text.lower()[:sequence_length])
print(text.lower()[:sequence_length])
decode_sequence(encode_text)

machine learning is a field of i
1/1 [==============================] - 0s 28ms/step


'wwwwwwwwwww'